<p><font size="6" color='grey'> <b>

Generative KI. Verstehen. Anwenden. Gestalten.
</b></font> </br></p>

<p><font size="5" color='grey'> <b>
MCP - Model Context Protocol mit LangChain Agent
</b></font> </br></p>

---

In [ ]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/GenAI.git#subdirectory=04_modul
!uv pip install --system -q fastmcp langchain-mcp-adapters nest_asyncio uvicorn

from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint
import nest_asyncio

nest_asyncio.apply()  # Wichtig für asyncio in Jupyter

setup_api_keys(['OPENAI_API_KEY'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()

# 1 | Übersicht

---

<p><font color='black' size="5">
Was lernst du in diesem Notebook?
</font></p>

**Nach diesem Notebook kannst du:**

1. ✅ **MCP-Server erstellen** - Eigene Tools mit FastMCP definieren
2. ✅ **Tools bereitstellen** - Server über HTTP erreichbar machen
3. ✅ **Client verbinden** - LangChain mit MCP-Server verbinden
4. ✅ **Agent orchestrieren** - LLM nutzt Tools automatisch
5. ✅ **Production-Ready** - Echte Netzwerk-Kommunikation verstehen

**Voraussetzungen:**
- Grundkenntnisse in Python
- Verständnis von LLMs und Agents (siehe M04a/M04b)
- Optional: HTTP/REST-Grundlagen

**Zeitbedarf:** Ca. 30-45 Minuten

<p><font color='black' size="5">
Was ist dieses Notebook?
</font></p>

Dieses Notebook zeigt, wie man einen **echten MCP-Server** mit **LangChain Agents** verbindet:

1. **FastMCP Server** - Stellt mathematische Tools bereit
2. **LangChain MCP Client** - Verbindet sich über HTTP mit dem Server
3. **LangChain 1.0+ Agent** - Nutzt die MCP-Tools intelligent
4. **Praktische Beispiele** - Mathematische Berechnungen durchführen

**Was ist MCP (Model Context Protocol)?**

MCP ist ein **standardisiertes Protokoll**, das LLMs (Large Language Models) mit externen Tools und Datenquellen verbindet. Denke an MCP wie an **USB für AI** - ein universeller Standard für Tool-Integration.

**Warum MCP?**
- ✅ **Standardisiert**: Ein Protokoll für alle Tools
- ✅ **Wiederverwendbar**: Einmal erstellen, überall nutzen
- ✅ **Skalierbar**: Einfach neue Tools hinzufügen
- ✅ **Production-Ready**: Für echte Anwendungen gedacht

**Architektur:**
```
LangChain Agent (dein Code)
        ↓
MCP Client (langchain-mcp-adapters)
        ↓ HTTP POST (JSON-RPC 2.0)
MCP Server (FastMCP auf localhost:8000)
        ↓
Tools (add, multiply, subtract, divide)
```

<p><font color='black' size="5">
Wie funktioniert MCP?
</font></p>

MCP basiert auf einem **Client-Server-Modell** mit drei Hauptkomponenten:

**1. MCP Server**
- Stellt **Tools** (Funktionen) bereit
- Kann **Ressourcen** (Daten) anbieten
- Implementiert das MCP-Protokoll (JSON-RPC 2.0)
- Läuft als eigenständiger Prozess

**2. MCP Client**
- Verbindet sich mit einem oder mehreren Servern
- Lädt verfügbare Tools vom Server
- Übersetzt LLM-Anfragen in MCP-Protokoll
- Verwaltet die Kommunikation

**3. LLM/Agent**
- Entscheidet, welche Tools benötigt werden
- Orchestriert mehrere Tool-Aufrufe
- Generiert natürlichsprachliche Antworten
- Nutzt Tools über den Client

**Transport-Optionen:**
- `stdio`: Standard Input/Output (für CLI-Tools)
- `streamable_http`: HTTP-basiert (für Web-Services) ✅ In diesem Notebook
- `sse`: Server-Sent Events
- `websocket`: WebSocket-Verbindung

<p><font color='black' size="5">
Praktisches Beispiel: Was passiert?
</font></p>

Stell dir vor, du fragst einen AI-Assistenten: **Berechne (3+5) ✕ 12**

**Ohne MCP:**
```
AI: "Das Ergebnis ist ungefähr 96."
```
→ AI rät oder rechnet im *Kopf* (fehleranfällig)

**Mit MCP:**
```
1. AI: "Ich brauche das add-Tool"
2. MCP Client → Server: add(3, 5)
3. Server → Client: 8
4. AI: "Ich brauche das multiply-Tool"
5. MCP Client → Server: multiply(8, 12)
6. Server → Client: 96
7. AI: "Das Ergebnis ist 96."
```
→ AI nutzt echte Tools (präzise, nachvollziehbar)

**Der Vorteil:**
- ✅ Präzise Ergebnisse (keine Halluzinationen)
- ✅ Erweiterbar (neue Tools einfach hinzufügen)
- ✅ Wiederverwendbar (gleicher Server für viele AIs)
- ✅ Nachvollziehbar (jeder Tool-Call ist dokumentiert)

<p><font color='black' size="5">
Notebook-Roadmap
</font></p>

Dieses Notebook ist in **5 praktische Schritte** unterteilt:

**📚 Kapitel 1: Übersicht** _(du bist hier)_
- Was ist MCP?
- Wie funktioniert es?
- Praktisches Beispiel

**🚀 Kapitel 2: MCP-Server erstellen**
- FastMCP Server definieren
- Mathematische Tools implementieren
- Server starten und testen

**🔌 Kapitel 3: Client verbinden**
- LangChain MCP Client konfigurieren
- Tools vom Server laden
- Verbindung prüfen

**🤖 Kapitel 4: LangChain Agent**
- Agent mit MCP-Tools erstellen
- LangChain 1.0+ Best Practices
- Agent orchestrieren

**💡 Kapitel 5: Praktische Beispiele**
- Einfache Berechnungen
- Komplexe Multi-Tool-Queries
- Debugging mit direkten HTTP-Calls

**✏️ Aufgabe: Eigene Tools**
- Erweitere den Server mit neuen Tools
- Teste deine Implementierung

**💡 Tipp:** Führe die Zellen nacheinander aus. Jeder Schritt baut auf dem vorherigen auf!

# 2 | FastMCP HTTP-Server starten

---

<p><font color='black' size="5">
Server-Datei erstellen
</font></p>

Wir erstellen eine **separate Python-Datei** für den FastMCP-Server, die wir dann als **subprocess** starten können.

In [ ]:
#@title 🚀 MCP HTTP-Server erstellen und starten{ display-mode: "form" }

import subprocess
import time
import sys
import os

mprint("### 🚀 MCP HTTP-Server erstellen")
mprint("---")

# SCHRITT 1: Server-Datei erstellen
server_code = '''
"""FastMCP Math Server - HTTP Transport"""
from fastmcp import FastMCP

# FastMCP Server initialisieren
mcp = FastMCP("MathTools")

@mcp.tool()
def add(a: int, b: int) -> int:
    """Addiert zwei Zahlen."""
    return a + b

@mcp.tool()
def multiply(a: int, b: int) -> int:
    """Multipliziert zwei Zahlen."""
    return a * b

@mcp.tool()
def subtract(a: int, b: int) -> int:
    """Subtrahiert b von a."""
    return a - b

@mcp.tool()
def divide(a: float, b: float) -> float:
    """Teilt a durch b."""
    if b == 0:
        raise ValueError("Division durch 0 ist nicht erlaubt")
    return a / b

if __name__ == "__main__":
    # Server mit HTTP Transport starten
    mcp.run(transport="http", host="127.0.0.1", port=8000, path="/mcp")
'''

# Server-Datei schreiben
with open('math_mcp_server.py', 'w', encoding='utf-8') as f:
    f.write(server_code)

print("✅ Server-Datei erstellt: math_mcp_server.py")
print("   • 4 Tools: add, multiply, subtract, divide\n")

# SCHRITT 2: Server als subprocess starten
print("⏳ Starte Server als subprocess...")

# Prüfe ob Server bereits läuft und beende ihn
try:
    import requests
    requests.get("http://127.0.0.1:8000", timeout=1)
    print("⚠️ Server läuft bereits, versuche zu beenden...")
    # Warte kurz und versuche erneut
    time.sleep(2)
except:
    pass  # Server läuft nicht, gut!

# Server starten
server_process = subprocess.Popen(
    [sys.executable, "math_mcp_server.py"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

# Warte bis Server bereit ist
print("⏳ Warte auf Server-Start...")
max_retries = 10
for i in range(max_retries):
    try:
        import requests
        response = requests.post(
            "http://127.0.0.1:8000/mcp",
            json={"jsonrpc": "2.0", "method": "tools/list", "id": 1},
            timeout=2
        )
        if response.status_code == 200:
            break
    except:
        time.sleep(1)
        if i == max_retries - 1:
            print("❌ Server konnte nicht gestartet werden!")
            server_process.kill()
            raise Exception("Server-Start fehlgeschlagen")

mprint("\n✅ MCP-Server läuft auf http://127.0.0.1:8000/mcp")
print(f"💡 Server-Prozess PID: {server_process.pid}")
print("💡 Der Server läuft als echter subprocess!")

<p><font color='black' size="5">
Server-Status testen
</font></p>

Wir können den Server direkt über HTTP testen:

In [ ]:
#@title 🧪 Server-Health-Check{ display-mode: "form" }

import requests
import json

mprint("### 🧪 Server Health Check")
mprint("---")

try:
    # Test-Request an Server
    response = requests.post(
        "http://127.0.0.1:8000/mcp",
        json={
            "jsonrpc": "2.0",
            "method": "tools/list",
            "id": 1
        },
        timeout=5
    )

    if response.status_code == 200:
        data = response.json()
        tools_count = len(data.get('result', {}).get('tools', []))
        print(f"✅ Server antwortet korrekt")
        print(f"📊 Verfügbare Tools: {tools_count}")
        print(f"🌐 URL: http://127.0.0.1:8000/mcp")
    else:
        print(f"⚠️ Server antwortet mit Status {response.status_code}")

except Exception as e:
    print(f"❌ Server nicht erreichbar: {e}")
    print("💡 Führe die vorherige Zelle erneut aus")

# 3 | LangChain MCP Client

---

<p><font color='black' size="5">
Client mit streamable_http Transport erstellen
</font></p>

Der **MultiServerMCPClient** verbindet sich über **streamable_http** mit dem Server:

**Wichtig:** Der Transport heißt `streamable_http`, nicht `http`!

In [ ]:
#@title 🔌 MCP Client mit HTTP initialisieren{ display-mode: "form" }

from langchain_mcp_adapters.client import MultiServerMCPClient

mprint("### 🔌 MCP Client Setup (HTTP)")
mprint("---")

# Client konfigurieren mit streamable_http Transport (nicht "http"!)
client = MultiServerMCPClient(
    {
        "math": {
            "transport": "streamable_http",  # ✅ Richtiger Transport-Name!
            "url": "http://127.0.0.1:8000/mcp"  # HTTP-Endpoint
        }
    }
)

print("✅ Client konfiguriert")
print(f"   Transport: streamable_http")
print(f"   Server-URL: http://127.0.0.1:8000/mcp")

<p><font color='black' size="5">
Tools vom Server laden
</font></p>

Der Client lädt jetzt die Tools über **streamable_http** vom Server:

In [ ]:
#@title 🔧 Tools vom Server laden (streamable_http)  { display-mode: "form" }

mprint("### 🔧 Tools vom MCP-Server laden")
mprint("---")

# Tools asynchron laden über streamable_http
tools = await client.get_tools()

print(f"✅ {len(tools)} Tools geladen vom Server:")
for tool in tools:
    print(f"   • {tool.name}: {tool.description}")

mprint("\n💡 Kommunikation erfolgt über streamable_http (JSON-RPC 2.0)!")

# 4 | LangChain 1.0+ Agent

---

<p><font color='black' size="5">
Agent mit MCP-Tools erstellen
</font></p>

Jetzt erstellen wir einen **LangChain 1.0+ Agent** mit den MCP-Tools. Der Agent kommuniziert über **streamable_http** mit dem Server!

In [ ]:
#@title 🤖 LangChain 1.0+ Agent erstellen{ display-mode: "form" }

from langchain.chat_models import init_chat_model
from langchain.agents import create_agent

mprint("### 🤖 LangChain 1.0+ Agent erstellen")
mprint("---")

# SCHRITT 1: LLM initialisieren (LangChain 1.0+ API)
print("1️⃣ LLM initialisieren...")
llm = init_chat_model("openai:gpt-4o-mini", temperature=0.0)
print("   ✅ GPT-4o-mini geladen\n")

# SCHRITT 2: Agent erstellen mit MCP-Tools
print("2️⃣ Agent erstellen...")
agent = create_agent(
    model=llm,
    tools=tools,  # Tools vom MCP-Server (über streamable_http)
    system_prompt="""Du bist ein hilfreicher Mathe-Assistent.
    Du hast Zugriff auf folgende Rechen-Tools über einen MCP HTTP-Server:
    - add: Addiert zwei Zahlen
    - multiply: Multipliziert zwei Zahlen
    - subtract: Subtrahiert zwei Zahlen
    - divide: Teilt zwei Zahlen

    Nutze diese Tools, um komplexe Berechnungen durchzuführen.
    Antworte immer auf Deutsch und erkläre deine Rechenschritte.""",
    debug=False  # Weniger Output
)
print("   ✅ Agent bereit\n")

mprint("🎯 Agent ist bereit für Queries!")
mprint("💡 Der Agent kommuniziert über streamable_http mit dem MCP-Server")

# 5 | Praktische Beispiele

---

<p><font color='black' size="5">
Einfache Berechnungen
</font></p>

In [ ]:
#@title 🤖 Query 1: Addition (3 + 5){ display-mode: "form" }

query = "Berechne 3 + 5"

mprint(f"### 🤖 Query: {query}")
mprint("---")

# Agent aufrufen
response = await agent.ainvoke({
    "messages": [{"role": "user", "content": query}]
})

# Antwort extrahieren
final_message = response["messages"][-1]
mprint(f"\n🤖 **Antwort:** {final_message.content}")

In [ ]:
#@title 🤖 Query 2: Multiplikation (12 * 24){ display-mode: "form" }

query = "Was ist 12 mal 24?"

mprint(f"### 🤖 Query: {query}")
mprint("---")

response = await agent.ainvoke({
    "messages": [{"role": "user", "content": query}]
})

final_message = response["messages"][-1]
mprint(f"\n🤖 **Antwort:** {final_message.content}")

<p><font color='black' size="5">
Komplexe Berechnungen (Multi-Tool)
</font></p>

Der Agent orchestriert mehrere HTTP-Calls zum Server:

In [ ]:
#@title 🤖 Query 3: Komplexe Berechnung (3+5)*12*2{ display-mode: "form" }

query = "Berechne (3+5)*12*2 mit den MCP-Mathe-Tools."

mprint(f"### 🤖 Query: {query}")
mprint("---")
print("💡 Erwarteter Ablauf (3 HTTP-Calls zum Server):")
print("   1. HTTP POST: add(3, 5) = 8")
print("   2. HTTP POST: multiply(8, 12) = 96")
print("   3. HTTP POST: multiply(96, 2) = 192\n")

response = await agent.ainvoke({
    "messages": [{"role": "user", "content": query}]
})

final_message = response["messages"][-1]
mprint(f"\n🤖 **Antwort:** {final_message.content}")

In [ ]:
#@title 🤖 Query 4: Division und Subtraktion{ display-mode: "form" }

query = "Berechne (100 - 20) / 4"

mprint(f"### 🤖 Query: {query}")
mprint("---")
print("💡 Erwarteter Ablauf:")
print("   1. HTTP POST: subtract(100, 20) = 80")
print("   2. HTTP POST: divide(80, 4) = 20.0\n")

response = await agent.ainvoke({
    "messages": [{"role": "user", "content": query}]
})

final_message = response["messages"][-1]
mprint(f"\n🤖 **Antwort:** {final_message.content}")

# 6 | HTTP-Kommunikation prüfen

---

<p><font color='black' size="5">
Direkter HTTP-Call zum Server
</font></p>

Wir können auch direkt HTTP-Requests an den Server senden:

In [ ]:
#@title 🔍 Direkter HTTP-Call testen{ display-mode: "form" }

import requests

mprint("### 🔍 Direkter HTTP-Call zum MCP-Server")
mprint("---")

# Tool-Call über HTTP
request_data = {
    "jsonrpc": "2.0",
    "method": "tools/call",
    "params": {
        "name": "multiply",
        "arguments": {"a": 7, "b": 8}
    },
    "id": 1
}

print("📤 Sende HTTP POST Request:")
print(f"   URL: http://127.0.0.1:8000/mcp")
print(f"   Method: tools/call")
print(f"   Tool: multiply(7, 8)\n")

response = requests.post(
    "http://127.0.0.1:8000/mcp",
    json=request_data,
    timeout=5
)

print(f"📥 Response Status: {response.status_code}")
print(f"📊 Ergebnis: {response.json()}\n")

mprint("💡 **Bestätigung:**")
print("Der MCP-Server läuft als ECHTER HTTP-Server!")
print("Kommunikation erfolgt über HTTP POST (JSON-RPC 2.0)")
print("Der Server ist von außerhalb erreichbar (localhost:8000)")

In [ ]:
#@title 🛑 MCP-Server beenden{ display-mode: "form" }

mprint("### 🛑 MCP-Server Cleanup")
mprint("---")

try:
    # Versuche den subprocess zu beenden
    if 'server_process' in globals():
        server_process.terminate()
        server_process.wait(timeout=5)
        print(f"✅ Server-Prozess (PID {server_process.pid}) wurde beendet")
    else:
        print("⚠️ Keine server_process Variable gefunden")
except Exception as e:
    print(f"⚠️ Fehler beim Beenden: {e}")
    print("💡 Server läuft möglicherweise noch. Kernel neu starten empfohlen.")

<p><font color='black' size="5">
Eigene MCP-Tools hinzufügen
</font></p>

**Ziel:** Erweitere den MCP-Server mit neuen Tools.

**Aufgabe 1: Potenzrechnung**
Füge ein `power(a, b)` Tool hinzu (bearbeite `math_mcp_server.py`, dann Server neu starten).

**Aufgabe 2: Modulo-Operation**
Füge ein `modulo(a, b)` Tool hinzu.

**Aufgabe 3: Komplexe Query**
Teste mit: `"Berechne (5 hoch 2) modulo 7"`

**Schritte:**
1. Bearbeite die Server-Datei `math_mcp_server.py`
2. Starte Server neu (führe Cleanup-Zelle aus, dann Zelle 2 neu)
3. Teste mit eigenen Queries

**Bonus:** Teste den Server von einem anderen Terminal mit `curl`:
```bash
curl -X POST http://localhost:8000/mcp \
  -H "Content-Type: application/json" \
  -d '{"jsonrpc":"2.0","method":"tools/list","id":1}'
```

# 8 | Zusammenfassung

---

<p><font color='black' size="5">
Was haben wir gelernt?
</font></p>

**In diesem Notebook haben wir:**

1. ✅ **MCP-Server erstellt** - Mit FastMCP und 4 mathematischen Tools
2. ✅ **HTTP-Kommunikation** - Server über `streamable_http` bereitgestellt
3. ✅ **Client verbunden** - Mit `MultiServerMCPClient` und Tools geladen
4. ✅ **LangChain 1.0+ Agent** - Mit `init_chat_model()` und `create_agent()`
5. ✅ **Praktische Anwendungen** - Komplexe Berechnungen durchgeführt
6. ✅ **Debugging-Skills** - Direkte HTTP-Calls zum Server getestet

**Wichtige Konzepte:**

**MCP (Model Context Protocol)**
- Standardisiertes Protokoll für Tool-Integration
- Client-Server-Architektur
- JSON-RPC 2.0 für Kommunikation

**FastMCP**
- Python-Bibliothek für MCP-Server
- Einfache Tool-Definition mit `@mcp.tool()`
- Unterstützt HTTP, stdio, SSE, WebSocket

**LangChain Integration**
- `langchain-mcp-adapters` für Client-Integration
- Seamless Integration mit LangChain Agents
- Tools werden automatisch als LangChain Tools verfügbar

**Best Practices:**
- Server als subprocess starten (nicht im Thread)
- Transport `streamable_http` für Web-Anwendungen
- Cleanup: Server sauber beenden
- Health Checks vor Client-Verbindung

**Anwendungsfälle für MCP:**

| Bereich | Beispiel-Tools |
|---------|---------------|
| **Datenbanken** | SQL-Queries, CRUD-Operationen |
| **Dateisystem** | Lesen, Schreiben, Suchen |
| **Web-APIs** | REST-Calls, Webhooks |
| **Cloud-Services** | AWS, Azure, GCP |
| **Entwicklung** | Git, CI/CD, Testing |

**Ressourcen:**
- [FastMCP Documentation](https://gofastmcp.com/deployment/running-server)
- [FastMCP GitHub](https://github.com/jlowin/fastmcp)
- [LangChain MCP Adapters](https://github.com/langchain-ai/langchain-mcp-adapters)
- [MCP Official Docs](https://modelcontextprotocol.io/)

**Nächste Schritte:**
- 🔧 Eigene Tools implementieren (z.B. Datei-System, APIs)
- 🌐 MCP-Server deployen (Docker, Cloud)
- 🔐 Security hinzufügen (API Keys, Rate Limiting)
- 🔄 Multiple Server orchestrieren

# 7 | Zusammenfassung

---

<p><font color='black' size="5">
Was haben wir gelernt?
</font></p>

**In diesem Notebook haben wir:**

1. ✅ **Echten MCP HTTP-Server** mit FastMCP erstellt
2. ✅ **HTTP-Client-Server-Kommunikation** (JSON-RPC 2.0)
3. ✅ **LangChain 1.0+ Agent** mit `init_chat_model()` und `create_agent()`
4. ✅ **MultiServerMCPClient** mit `streamable_http` Transport
5. ✅ **Praktische Queries** mit Multi-Tool-Orchestrierung
6. ✅ **Echte Netzwerk-Kommunikation** über HTTP POST

**Wichtiger Hinweis:**
- Der Transport heißt `streamable_http`, nicht `http`!
- Unterstützte Transports: `stdio`, `sse`, `websocket`, `streamable_http`

**Warum streamable_http statt stdio?**

| Feature | stdio | streamable_http (dieses Notebook) |
|---------|-------|-----------------------------------|
| **Jupyter/Colab** | ❌ Nicht unterstützt | ✅ Funktioniert |
| **Remote-Server** | ❌ Nur lokal | ✅ Über Netzwerk |
| **Debugging** | Schwieriger | ✅ Einfach (curl, Postman) |
| **Production** | Möglich | ✅ Standard |

**Ressourcen:**
- [FastMCP Documentation](https://www.prefect.io/fastmcp)
- [LangChain MCP Adapters](https://github.com/langchain-ai/langchain-mcp-adapters)
- [MCP Official Docs](https://modelcontextprotocol.io/)

**Nächste Schritte:**
- 🌐 Deployment auf echtem Server (AWS, GCP, Azure)
- 🔐 Authentication und Security (API Keys)
- 🐳 Docker-Containerisierung
- 🔄 Multi-Server-Integration